In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, auc

BATCH_SIZE = 32
# IMAGE_SIZE = (90, 90)
#IMAGE_SIZE = (50, 50)
IMAGE_SIZE = (40, 40)

def get_dataset_binaire():
    return keras.utils.image_dataset_from_directory(
        "Datasets/Dataset binaire/",
        labels="inferred",
        label_mode='binary',
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=IMAGE_SIZE,
        shuffle=True,
        seed=42,
        validation_split=0.2,
        subset="both",
        interpolation='area' #area pour rétrecir / bicubic pour élargir
    )

def get_dataset_multiclasse():
    return keras.utils.image_dataset_from_directory(
        "Datasets/Dataset multiclasse/",
        labels="inferred",
        label_mode='int',
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=IMAGE_SIZE,
        shuffle=True,
        seed=42,
        validation_split=0.2,
        subset="both",
        interpolation='area' #area pour rétrecir / bicubic pour élargir
    )

def DrawConfusionMatrix(y, y_pred, labels=None):
    cm = confusion_matrix(y, y_pred, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot()
    plt.show()

def DrawPrecisionRecallCurve(y_validation, y_val_pred):
    precision, recall, _ = precision_recall_curve(y_validation, y_val_pred)
    auc_score = auc(recall, precision)
    plt.figure(figsize=(8, 8))
    plt.plot(recall, precision, label=f'Precision-Recall curve (AUC = {auc_score:.2f})', color='b')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='best')
    plt.show()

In [ ]:
train_binaire, validation_binaire = get_dataset_binaire()
classes_binaire = train_binaire.class_names
print(classes_binaire)
train_multiclasse, validation_multiclasse = get_dataset_multiclasse()
classes_multiclasse = train_multiclasse.class_names
print(classes_multiclasse)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_binaire = train_binaire.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
train_multiclasse = train_multiclasse.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
images_train, labels_train, images_validation, labels_validation = [], [], [], []

for image_batch, label_batch in train_binaire:
    images_train.extend(image_batch.numpy())
    labels_train.extend(label_batch.numpy())

for image_batch, label_batch in validation_binaire:
    images_validation.extend(image_batch.numpy())
    labels_validation.extend(label_batch.numpy())

images_train = np.array(images_train)
images_validation = np.array(images_validation)

labels_train = np.array(labels_train)
labels_validation = np.array(labels_validation)

X_train = images_train.reshape(images_train.shape[0], -1)
y_train = labels_train.ravel()

X_validation = images_validation.reshape(images_validation.shape[0], -1)
y_validation = labels_validation.ravel()

print("X shape : {}".format(str(X_train.shape)))
print("Y shape : {}".format(str(y_train.shape)))
print("X shape : {}".format(str(X_validation.shape)))
print("Y shape : {}".format(str(y_validation.shape)))

In [ ]:
plt.figure(figsize=(8, 8))
for images, labels in train_binaire.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(labels[i].numpy().astype(int)[0])
        plt.axis("off")

plt.figure(figsize=(8, 8))
for images, labels in train_multiclasse.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(labels[i].numpy().astype(int))
        plt.axis("off")

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(X_train, y_train)

In [ ]:
y_val_pred = clf.predict(X_validation)

DrawPrecisionRecallCurve(y_validation, y_val_pred)

print("score on train: "+ str(clf.score(X_train, y_train)))
print("score on test: " + str(clf.score(X_validation, y_validation)))
DrawConfusionMatrix(y_validation, y_val_pred, svm.classes_)